In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
%matplotlib inline

In [ ]:
pd.options.display.max_rows= 81
missing_percentage = traindf.isnull().sum()/len(traindf)*100
missing_percentage

In [ ]:
pd.options.display.max_rows= 81
missing_percentage = testdf.isnull().sum()/len(testdf)*100
missing_percentage

In [ ]:
missing_columns_train = traindf.loc[:,traindf.isnull().any()].columns
missing_columns_train

In [ ]:
missing_value_df_train = traindf.loc[:,traindf.isnull().any()]
missing_columns_train = missing_value_df_train.columns
missing_columns_train

In [ ]:
import random as rand
lot_mean = traindf["LotFrontage"].mean()
lot_sd = traindf["LotFrontage"].std()
random_numbers = list()
for i in range(259):
    random_numbers.append(rand.uniform(lot_mean-3*lot_sd,lot_mean+3*lot_sd))
traindf.loc[traindf["LotFrontage"].isnull(),"LotFrontage"] = random_numbers

In [ ]:
sns.distplot(testdf["LotFrontage"])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
cat_columns_train = list(traindf.select_dtypes("object").columns)
cat_columns_train = [x for x in cat_columns_train if x!= "FireplaceQu"]
transformer = ColumnTransformer(transformers=[("L",OrdinalEncoder(),cat_columns_train)],remainder='passthrough')
cat_columns_imputed_train = transformer.fit_transform(traindf[cat_columns_train])
traindf[cat_columns_train] = cat_columns_imputed_train
cat_columns_test = list(testdf.select_dtypes("object").columns)
cat_columns_test = [x for x in cat_columns_test if x!= "FireplaceQu"]
transformer = ColumnTransformer(transformers=[("L",OrdinalEncoder(),cat_columns_test)],remainder='passthrough')
cat_columns_imputed_test = transformer.fit_transform(testdf[cat_columns_test])
testdf[cat_columns_test] = cat_columns_imputed_test
encoder = OrdinalEncoder()
non_null_train = traindf["FireplaceQu"].dropna()
encoded_train = encoder.fit_transform(np.array(non_null_train).reshape(-1,1))
traindf.loc[traindf["FireplaceQu"].notnull(),"FireplaceQu"] = np.squeeze(encoded_train)

In [ ]:
!pip install fancyimpute
from fancyimpute import KNN
imputer = KNN(k=5)
FireplaceQu_imputed = imputer.fit_transform(np.array(traindf["FireplaceQu"]).reshape(-1,1))
traindf["FireplaceQu"] = FireplaceQu_imputed

In [ ]:
imputer = KNN(k=5)
FireplaceQu_imputed = imputer.fit_transform(np.array(testdf["FireplaceQu"]).reshape(-1,1))
testdf["FireplaceQu"] = FireplaceQu_imputed

In [ ]:
X = traindf.iloc[:,:75]
y = traindf["SalePrice"]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
model0 = DecisionTreeRegressor()
model0.fit(X_train,y_train)
y_pred = model0.predict(X_test)
mae_0 = mean_absolute_error(y_test,y_pred)
rmse_0 = np.sqrt(mean_squared_error(y_test,y_pred))
r_squared_0 = r2_score(y_test,y_pred)
print("Root mean square error : ",rmse_0)
print("Mean absolute error : ",mae_0)
print("r squared : ",r_squared_0)

In [ ]:
model1 = RandomForestRegressor(n_estimators=100,random_state=2)
model1.fit(X_train,y_train)
y_pred = model1.predict(X_test)
mae_1 = mean_absolute_error(y_test,y_pred)
rmse_1 = np.sqrt(mean_squared_error(y_test,y_pred))
r_squared_1 = r2_score(y_test,y_pred)
print("Root mean square error : ",rmse_1)
print("Mean absolute error : ",mae_1)
print("r squared : ",r_squared_1)

In [ ]:
model2 = LinearRegression()
model2.fit(X_train,y_train)
y_pred = model2.predict(X_test)
mae_2 = mean_absolute_error(y_test,y_pred)
rmse_2 = np.sqrt(mean_squared_error(y_test,y_pred))
r_squared_2 = r2_score(y_test,y_pred)
print("Root mean square error : ",rmse_2)
print("Mean absolute error : ",mae_2)
print("r squared : ",r_squared_2)

In [ ]:
from xgboost import XGBRegressor
model3 = XGBRegressor(eta=0.1)
model3.fit(X_train,y_train)
y_pred = model3.predict(X_test)
mae_3 = mean_absolute_error(y_test,y_pred)
rmse_3 = np.sqrt(mean_squared_error(y_test,y_pred))
r_squared_3 = r2_score(y_test,y_pred)
print("Root mean square error : ",rmse_3)
print("Mean absolute error : ",mae_3)
print("r squared : ",r_squared_3)

In [ ]:
plotdf = pd.DataFrame({"Models":["Decision Tree","Random Forest","Linear Regression","XGBoost Regressor"],
                      "RMSE":[rmse_0,rmse_1,rmse_2,rmse_3],"MAE":[mae_0,mae_1,mae_2,mae_3],
                      "R-Squared":[r_squared_0,r_squared_1,r_squared_2,r_squared_3]})
plotdf

In [ ]:
y_pred_test = model3.predict(testdf)
y_pred_test